In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
from medvqa.models.vision.visual_modules import create_detectron2_model

In [3]:
model = create_detectron2_model("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml", verbose=False)

Building Detectron2 model
Loading weights from https://dl.fbaipublicfiles.com/detectron2/COCO-Detection/faster_rcnn_R_50_FPN_3x/137849458/model_final_280758.pkl
Detectron2 model successfully built


In [4]:
# model

In [5]:
from importlib import reload
import  medvqa

In [6]:
# reload(medvqa.datasets.image_processing)

In [7]:
from detectron2.config import get_cfg
from detectron2 import model_zoo
from detectron2.data import MetadataCatalog, DatasetCatalog, build_detection_train_loader, build_detection_test_loader

from medvqa.datasets.chest_imagenome import CHEST_IMAGENOME_BBOX_NAMES
from medvqa.datasets.mimiccxr.mimiccxr_vision_dataset_management import Detectron2AdaptedDataset

In [8]:
from medvqa.datasets.image_processing import image_size_cache

In [9]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))

In [10]:
for d in ['train', 'validate']:
    DatasetCatalog.register('chest_imagenome_' + d, lambda d=d: Detectron2AdaptedDataset(d, None))
    MetadataCatalog.get('chest_imagenome_' + d).set(thing_classes=CHEST_IMAGENOME_BBOX_NAMES)
cfg.DATASETS.TRAIN = ('chest_imagenome_train',)
cfg.DATASETS.TEST = ('chest_imagenome_validate',)

In [10]:
_d = DatasetCatalog.get('chest_imagenome_train')

Loading cached detailed metadata from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/detailed_metadata.pkl
Loaded 240530 non-gold DICOM IDs from Chest Imagenome
Using source image size mode: medium_512
Loaded 242280 decent image IDs


  0%|          | 0/227835 [00:00<?, ?it/s]

image_size_cache loaded from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/image_size_cache.pkl


100%|██████████| 227835/227835 [00:03<00:00, 65508.44it/s]

Skipped 992 images


In [11]:
_d_val = DatasetCatalog.get('chest_imagenome_validate')

  0%|          | 0/227835 [00:00<?, ?it/s]

Loading cached detailed metadata from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/detailed_metadata.pkl
Loaded 240530 non-gold DICOM IDs from Chest Imagenome
Using source image size mode: medium_512
Loaded 242280 decent image IDs


100%|██████████| 227835/227835 [00:00<00:00, 682504.91it/s]

Skipped 11 images


In [12]:
len(_d_val)

1934

In [13]:
print(cfg)

CUDNN_BENCHMARK: False
DATALOADER:
  ASPECT_RATIO_GROUPING: True
  FILTER_EMPTY_ANNOTATIONS: True
  NUM_WORKERS: 4
  REPEAT_THRESHOLD: 0.0
  SAMPLER_TRAIN: TrainingSampler
DATASETS:
  PRECOMPUTED_PROPOSAL_TOPK_TEST: 1000
  PRECOMPUTED_PROPOSAL_TOPK_TRAIN: 2000
  PROPOSAL_FILES_TEST: ()
  PROPOSAL_FILES_TRAIN: ()
  TEST: ('chest_imagenome_validate',)
  TRAIN: ('chest_imagenome_train',)
GLOBAL:
  HACK: 1.0
INPUT:
  CROP:
    ENABLED: False
    SIZE: [0.9, 0.9]
    TYPE: relative_range
  FORMAT: BGR
  MASK_FORMAT: polygon
  MAX_SIZE_TEST: 1333
  MAX_SIZE_TRAIN: 1333
  MIN_SIZE_TEST: 800
  MIN_SIZE_TRAIN: (640, 672, 704, 736, 768, 800)
  MIN_SIZE_TRAIN_SAMPLING: choice
  RANDOM_FLIP: horizontal
MODEL:
  ANCHOR_GENERATOR:
    ANGLES: [[-90, 0, 90]]
    ASPECT_RATIOS: [[0.5, 1.0, 2.0]]
    NAME: DefaultAnchorGenerator
    OFFSET: 0.0
    SIZES: [[32], [64], [128], [256], [512]]
  BACKBONE:
    FREEZE_AT: 2
    NAME: build_resnet_fpn_backbone
  DEVICE: cuda
  FPN:
    FUSE_TYPE: sum
    IN_FE

In [14]:
cfg.INPUT

CfgNode({'MIN_SIZE_TRAIN': (640, 672, 704, 736, 768, 800), 'MIN_SIZE_TRAIN_SAMPLING': 'choice', 'MAX_SIZE_TRAIN': 1333, 'MIN_SIZE_TEST': 800, 'MAX_SIZE_TEST': 1333, 'RANDOM_FLIP': 'horizontal', 'CROP': CfgNode({'ENABLED': False, 'TYPE': 'relative_range', 'SIZE': [0.9, 0.9]}), 'FORMAT': 'BGR', 'MASK_FORMAT': 'polygon'})

In [11]:
from detectron2.data import DatasetMapper

In [16]:
# if "SemanticSegmentor" in cfg.MODEL.META_ARCHITECTURE:
#             mapper = DatasetMapper(cfg, is_train=True, augmentations=build_sem_seg_train_aug(cfg))
#         else:
#             mapper = None
#         return build_detection_train_loader(cfg, mapper=mapper

dataloader = build_detection_train_loader(
    dataset=DatasetCatalog.get('chest_imagenome_train'),
    mapper=DatasetMapper(is_train=True, image_format=cfg.INPUT.FORMAT, augmentations=[]),
    total_batch_size=cfg.SOLVER.IMS_PER_BATCH,
    num_workers=4,
)

In [12]:
dataloader_val = build_detection_test_loader(
    dataset=DatasetCatalog.get('chest_imagenome_validate'),
    mapper=DatasetMapper(is_train=False, image_format=cfg.INPUT.FORMAT, augmentations=[]),
    num_workers=4,
)

Loading cached detailed metadata from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/mimiccxr/detailed_metadata.pkl
Loaded 240530 non-gold DICOM IDs from Chest Imagenome
Using source image size mode: medium_512
Loaded 242280 decent image IDs


  0%|          | 0/227835 [00:00<?, ?it/s]

image_size_cache loaded from /mnt/data/pamessina/workspaces/medvqa-workspace/cache/image_size_cache.pkl


100%|██████████| 227835/227835 [00:00<00:00, 471135.47it/s]

Skipped 11 images


In [13]:
batch = next(iter(dataloader_val))

In [14]:
len(batch)

1

In [15]:
batch[0]

{'file_name': '/mnt/workspace/mimic-cxr-jpg/images-medium(512)/p10/p10003502/s52139270/550e6f3b-f008c1d0-8d2dee2a-649b30f4-101a98cc.jpg',
 'image_id': '550e6f3b-f008c1d0-8d2dee2a-649b30f4-101a98cc',
 'height': 615,
 'width': 512,
 'image': tensor([[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],
 
         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],
 
         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]]], dtype=torch.uint8)}

In [16]:
batch[0]['image'].shape

torch.Size([3, 615, 512])

In [18]:
from detectron2.utils.events import EventStorage

In [19]:
with EventStorage() as storage:
    model.eval()
    output = model(batch)

In [20]:
output[0]['instances']

Instances(num_instances=8, image_height=615, image_width=512, fields=[pred_boxes: Boxes(tensor([[ 59.9608,  31.3847, 450.1136, 504.6334],
        [ 71.6471,  13.2026, 451.3328, 496.2819],
        [131.7231,  12.1683, 348.4443, 251.0878],
        [ 44.5138,  34.2821, 468.4243, 481.5617],
        [150.4755,   8.9024, 348.4955, 274.0350],
        [129.8500,  16.7071, 347.4103, 236.9656],
        [ 48.3948,  48.5375, 468.9896, 478.5137],
        [258.0957,   7.2502, 468.6046, 457.0898]], device='cuda:0',
       grad_fn=<IndexBackward0>)), scores: tensor([0.6643, 0.5768, 0.3042, 0.2017, 0.1412, 0.0812, 0.0509, 0.0506],
       device='cuda:0', grad_fn=<IndexBackward0>), pred_classes: tensor([15,  0, 15, 59,  0, 16, 57,  0], device='cuda:0')])

In [11]:
!python ../train_visual_module.py \
    --epochs 60 \
    --batches-per-epoch 120 \
    --batch-size 50 \
    --num-workers 3 \
    --iters-to-accumulate 2 \
    --optimizer-name "adamw" \
    --scheduler "warmup+decay" \
    --lr 1e-6 \
    --warmup-and-decay-args "1e-6,7,2e-4,53,1e-6" \
    --use-mimiccxr \
    --mimiccxr-weight 1.0 \
    --mimiccxr-view-mode "chest_imagenome" \
    --predict-bboxes-chest-imagenome \
    --clamp-bboxes-chest-imagenome \
    --use-chest-imagenome-decent-images-only \
    --raw-image-encoding "detectron2" \
    --detectron2-model-yaml "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml" \
    --image-size 512 512 \
    --use-amp \
    --save

script's arguments:
   epochs: 60
   batches_per_epoch: 120
   checkpoint_folder: None
   visual_input_mode: raw-image
   raw_image_encoding: detectron2
   image_local_feat_size: 1024
   image_encoder_pretrained_weights_path: None
   freeze_image_encoder: False
   imagenet_pretrained: False
   visual_features_mlp_in_dim: None
   visual_features_mlp_out_dim: None
   visual_features_mlp_hidden_dims: None
   iuxray_precomputed_visual_features_path: None
   mimiccxr_precomputed_visual_features_path: None
   chexpert_precomputed_visual_features_path: None
   vinbig_precomputed_visual_features_path: None
   clip_version: None
   huggingface_model_name: None
   chest_imagenome_bbox_hidden_size: 128
   chest_imagenome_bbox_regressor_version: v1
   torchxrayvision_weights_name: None
   detectron2_model_yaml: COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml
   num_regions: None
   roi_heads_batch_size_per_image: 128
   rpn_batch_size_per_image: 128
   optimizer_name: adamw
   lr: 1e-06
   scheduler: 

loss 8.20738, d2box_loss 0.85440, d2cls_loss 3.43679, d2rpncls_loss 1.52046, d2rpnloc_loss 0.39105, 128.23 secs
(2) Validation stage ...
chestimgbbmf1 0.00000, chestimgbbiou 0.00000, chestimgbbmae 0.00000, 32.67 secs
Adjusting learning rate of group 0 to 2.1317e-06.
---- Epoch 2/60
(1) Training stage (lr = 0.000002) ...
loss 4.78592, d2box_loss 0.87551, d2cls_loss 2.18433, d2rpncls_loss 0.30223, d2rpnloc_loss 0.36458, 130.20 secs
(2) Validation stage ...
chestimgbbmf1 0.00000, chestimgbbiou 0.00000, chestimgbbmae 0.00000, 31.16 secs
Adjusting learning rate of group 0 to 4.5440e-06.
---- Epoch 3/60
(1) Training stage (lr = 0.000005) ...
loss 3.02361, d2box_loss 0.86415, d2cls_loss 1.36573, d2rpncls_loss 0.20689, d2rpnloc_loss 0.33931, 130.45 secs
(2) Validation stage ...
chestimgbbmf1 0.00000, chestimgbbiou 0.00000, chestimgbbmae 0.00000, 32.06 secs
Adjusting learning rate of group 0 to 9.6863e-06.
---- Epoch 4/60
(1) Training stage (lr = 0.000010) ...
loss 2.57191, d2box_loss 0.82839, 

In [17]:
!python ../train_visual_module.py \
    --checkpoint-folder "models/visual_module/20230301_144556_mim_D2-CocoDet-faster-rcnn-R-50-FPN-3x" \
    --epochs 57 \
    --batches-per-epoch 120 \
    --batch-size 50 \
    --num-workers 3 \
    --iters-to-accumulate 2 \
    --save

script's arguments:
   epochs: 57
   batches_per_epoch: 120
   checkpoint_folder: models/visual_module/20230301_144556_mim_D2-CocoDet-faster-rcnn-R-50-FPN-3x
   visual_input_mode: raw-image
   raw_image_encoding: densenet-121
   image_local_feat_size: 1024
   image_encoder_pretrained_weights_path: None
   freeze_image_encoder: False
   imagenet_pretrained: False
   visual_features_mlp_in_dim: None
   visual_features_mlp_out_dim: None
   visual_features_mlp_hidden_dims: None
   iuxray_precomputed_visual_features_path: None
   mimiccxr_precomputed_visual_features_path: None
   chexpert_precomputed_visual_features_path: None
   vinbig_precomputed_visual_features_path: None
   clip_version: None
   huggingface_model_name: None
   chest_imagenome_bbox_hidden_size: 128
   chest_imagenome_bbox_regressor_version: v1
   torchxrayvision_weights_name: None
   detectron2_model_yaml: None
   num_regions: None
   roi_heads_batch_size_per_image: 128
   rpn_batch_size_per_image: 128
   optimizer_name:

chestimgbbmf1 0.01111, chestimgbbiou 0.65831, chestimgbbmae 22.02673, 46.20 secs
Adjusting learning rate of group 0 to 2.0648e-05.
---- Epoch 5/60
(1) Training stage (lr = 0.000021) ...
loss 1.74694, d2box_loss 0.66081, d2cls_loss 0.53649, d2rpncls_loss 0.12732, d2rpnloc_loss 0.29629, 131.14 secs
(2) Validation stage ...
chestimgbbmf1 0.05654, chestimgbbiou 0.70886, chestimgbbmae 13.25749, 50.88 secs
Adjusting learning rate of group 0 to 4.4014e-05.
---- Epoch 6/60
(1) Training stage (lr = 0.000044) ...
loss 1.41344, d2box_loss 0.59881, d2cls_loss 0.52826, d2rpncls_loss 0.11207, d2rpnloc_loss 0.27557, 132.40 secs
(2) Validation stage ...
chestimgbbmf1 0.09624, chestimgbbiou 0.73046, chestimgbbmae 11.63401, 52.56 secs
Adjusting learning rate of group 0 to 9.3823e-05.
---- Epoch 7/60
(1) Training stage (lr = 0.000094) ...
loss 1.28725, d2box_loss 0.52570, d2cls_loss 0.41610, d2rpncls_loss 0.09158, d2rpnloc_loss 0.26064, 132.81 secs
(2) Validation stage ...
chestimgbbmf1 0.13710, chestimg

In [20]:
!python ../train_visual_module.py \
    --pretrained-checkpoint-folder-path "/mnt/data/pamessina/workspaces/medvqa-workspace/models/visual_module/20230301_144556_mim_D2-CocoDet-faster-rcnn-R-50-FPN-3x" \
    --epochs 40 \
    --batches-per-epoch 120 \
    --batch-size 30 \
    --num-workers 3 \
    --iters-to-accumulate 4 \
    --optimizer-name "adamw" \
    --scheduler "warmup+decay" \
    --lr 1e-6 \
    --warmup-and-decay-args "1e-6,5,3e-4,35,3e-6" \
    --use-mimiccxr \
    --mimiccxr-weight 1.0 \
    --mimiccxr-view-mode "chest_imagenome" \
    --predict-bboxes-chest-imagenome \
    --clamp-bboxes-chest-imagenome \
    --use-chest-imagenome-decent-images-only \
    --raw-image-encoding "detectron2" \
    --detectron2-model-yaml "COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml" \
    --image-size 512 512 \
    --use-amp \
    --save

script's arguments:
   epochs: 40
   batches_per_epoch: 120
   checkpoint_folder: None
   visual_input_mode: raw-image
   raw_image_encoding: detectron2
   image_local_feat_size: 1024
   image_encoder_pretrained_weights_path: None
   freeze_image_encoder: False
   imagenet_pretrained: False
   visual_features_mlp_in_dim: None
   visual_features_mlp_out_dim: None
   visual_features_mlp_hidden_dims: None
   iuxray_precomputed_visual_features_path: None
   mimiccxr_precomputed_visual_features_path: None
   chexpert_precomputed_visual_features_path: None
   vinbig_precomputed_visual_features_path: None
   clip_version: None
   huggingface_model_name: None
   chest_imagenome_bbox_hidden_size: 128
   chest_imagenome_bbox_regressor_version: v1
   torchxrayvision_weights_name: None
   detectron2_model_yaml: COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml
   num_regions: None
   roi_heads_batch_size_per_image: 128
   rpn_batch_size_per_image: 128
   optimizer_name: adamw
   lr: 1e-06
   scheduler: 

Checkpoint successfully loaded!
--------------------------------------------------
13) Defining log_metrics_handler ...
MetricsLogger :: we'll be logging to /mnt/data/pamessina/workspaces/medvqa-workspace/models/visual_module/20230301_180949_mim_D2-CocoDet-faster-rcnn-R-50-FPN-3x/metrics_logs.csv
--------------------------------------------------
14) Running trainer engine ...
---- Epoch 1/40
(1) Training stage (lr = 0.000001) ...
loss 1.03849, d2box_loss 0.35539, d2cls_loss 0.29932, d2rpncls_loss 0.05736, d2rpnloc_loss 0.17804, 76.87 secs
(2) Validation stage ...
chestimgbbmf1 0.35211, chestimgbbiou 0.81823, chestimgbbmae 6.89542, 44.86 secs
Adjusting learning rate of group 0 to 3.1291e-06.
---- Epoch 2/40
(1) Training stage (lr = 0.000003) ...
loss 0.82427, d2box_loss 0.35252, d2cls_loss 0.29779, d2rpncls_loss 0.05466, d2rpnloc_loss 0.17871, 76.21 secs
(2) Validation stage ...
chestimgbbmf1 0.35116, chestimgbbiou 0.81706, chestimgbbmae 6.94133, 46.33 secs
Adjusting learning rate of g

chestimgbbmf1 0.36892, chestimgbbiou 0.82152, chestimgbbmae 6.73778, 72.68 secs
Adjusting learning rate of group 0 to 2.1591e-05.
---- Epoch 26/40
(1) Training stage (lr = 0.000022) ...
loss 0.87882, d2box_loss 0.33809, d2cls_loss 0.28368, d2rpncls_loss 0.04809, d2rpnloc_loss 0.16404, 91.30 secs
(2) Validation stage ...
chestimgbbmf1 0.36788, chestimgbbiou 0.82033, chestimgbbmae 6.79036, 73.32 secs
Adjusting learning rate of group 0 to 1.8929e-05.
---- Epoch 27/40
(1) Training stage (lr = 0.000019) ...
loss 0.83756, d2box_loss 0.33865, d2cls_loss 0.28290, d2rpncls_loss 0.04819, d2rpnloc_loss 0.16352, 93.02 secs
(2) Validation stage ...
chestimgbbmf1 0.36990, chestimgbbiou 0.82275, chestimgbbmae 6.68623, 71.47 secs
Adjusting learning rate of group 0 to 1.6595e-05.
---- Epoch 28/40
(1) Training stage (lr = 0.000017) ...
loss 0.85930, d2box_loss 0.33967, d2cls_loss 0.28670, d2rpncls_loss 0.05582, d2rpnloc_loss 0.16885, 94.10 secs
(2) Validation stage ...
chestimgbbmf1 0.37111, chestimgbbi